# Extraction of relevant terms and parsing of list attributes

In [1]:
#TODO: remove after development 
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 5400)
pd.set_option('max_colwidth', 100)

## Loading data

In [4]:
import pickle

#seed_name = 'hair_dryer'
#seed_name = 'video_codec'
#seed_name = 'diesel'
seed_name = "contact_lens"
#seed_name = "contact_lens_us_c"
#seed_name = "3d_printer"

src_dir = "."

with open(src_dir + "/data/" + seed_name + "/cleaned_text.pkl", 'rb') as infile:
    df = pickle.load(infile)

In [5]:
print(df.shape)
df.head()

(3225, 15)


,pub_num,publication_number,country_code,family_id,priority_date,title_text,abstract_text,claims_text,refs,cpcs,ipcs,assignees_harmonized,ExpansionLevel,text_prep,tokens
0,4104187,US-4104187-A,US,24713901,19760412,Composition and method treating soft contact lenses at elevated temperatures,Method and compositions are provided for elevated temperature disinfection of soft contact lense...,What is claimed is: \n \n 1. A soft lens thermal disinfecting and cleaning solution co...,"US-3171752-A,US-4013576-A,US-2089212-A,US-3888782-A,US-3908680-A,US-3882036-A,,US-4046706-A","C11D1/523,C11D3/0078,A61L12/148","C11D17/00,G02C7/04,C11D1/66,C11D1/52,G02C13/00,C11D1/90,C11D3/20,A61L12/14,C11D3/00,C11D10/02,C1...",BARNES HIND PHARM INC,Seed,Composition and method treating soft contact lenses at elevated temperatures Method and composit...,"[composition, treating, soft, contact, lenses, elevated, temperatures, compositions, provided, e..."
1,6000534,US-6000534-A,US,24806490,19960816,Contact lens disinfecting device and disinfection system,The present invention provides a device for disinfecting and storing a contact lens comprising a...,I claim: \n \n 1. A device for disinfecting and storing a contact lens comprising: a c...,"US-4750610-A,US-4889693-A,US-5143104-A,US-5353949-A,US-5283053-A,US-4136796-A,US-4637919-A,US-47...","A45C11/005,Y10S134/901,A61L12/128","A61L12/12,G02C13/00,A61L2/18,A45C11/00",ALLERGAN SALES INC,Seed,Contact lens disinfecting device and disinfection system The present invention provides a device...,"[contact, lens, disinfecting, device, disinfection, system, device, disinfecting, storing, conta..."
2,7281794,US-7281794-B2,US,25536666,20011116,Disposable ophthalmic lens,"An ophthalmic lens with high definition, wide field of view and high magnification has, in one e...",What is claimed and desired to be secured by Letters Patent is: \n \n 1. An ophthalmic...,"US-6471396-B2,US-4921205-A,US-4613379-A,US-5706073-A,US-4728183-A",A61B3/125,"G02C7/04,A61B3/125",HEACOCK GREGORY L,Seed,"Disposable ophthalmic lens An ophthalmic lens with high definition, wide field of view and high ...","[disposable, ophthalmic, lens, ophthalmic, lens, high, definition, wide, view, high, magnificati..."
3,8693104,US-8693104-B2,US,44651943,20100824,Lens with compound linear-convex meniscus wall,The present invention relates generally to an arcuate liquid meniscus lens with a meniscus wall....,The invention claimed is: \n \n 1. An optical lens comprising:\n a front curve lens co...,"WO-2008062067-A1,US-7724444-B2,US-7311398-B2,WO-2004099844-A1,US-7327523-B2,WO-2005069043-A1,US-...","G02C7/085,G02B26/004,A61F2/1635,G02C7/08,G02B3/14,G02C7/04","G02B3/12,G02B1/06","KERNICK EDWARD R,RIALL JAMES DANIEL,JOHNSON & JOHNSON VISION CARE,OTTS DANIEL B,TONER ADAM,PUGH ...",Seed,Lens with compound linear-convex meniscus wall The present invention relates generally to an arc...,"[lens, compound, linear, convex, meniscus, wall, arcuate, liquid, meniscus, lens, meniscus, wall..."
4,8215770,US-8215770-B2,US,40337739,20070223,Ophthalmic dynamic aperture,Embodiments of the present invention relate to an electro-active element having a dynamic apertu...,"1. An ophthalmic device, comprising:\n an electro-active element comprising a mostly transparent...","US-6956682-B2,US-5608567-A,US-5963300-A,US-4904063-A,US-6955433-B1,US-6894751-B2,US-4423929-A,US...","G02C7/12,A61F2/145,A61F2250/001,A61F2/141,G02C2202/04,A61F2/16,A61F2/14,A61F2/1659,G02C7/083,A61...","G02C7/04,A61F2/16","HADDOCK JOSHUA N,VAN HEUGTEN ANTHONY,KOKONASKI WILLIAM,BLUM RONALD D,HUNKELER JOHN,E A OPHTHALMICS",Seed,Ophthalmic dynamic aperture Embodiments of the present invention relate to an electro-active ele...,"[ophthalmic, dynamic, aperture, embodiments, electro, active, element, dynamic, aperture, electr..."


## Extracting useful attributes

In [6]:
from datetime import datetime

def parse_date(date):
    date = str(date)
    try:
        datetime_object = datetime.strptime(date, '%Y%m%d')
    except ValueError:
        datetime_object = datetime.strptime("20191231", '%Y%m%d')
    return  datetime_object.year, datetime_object.month, datetime_object.day

In [7]:
df['year'], df['month'], df['day'] = zip(*df['priority_date'].map(parse_date))

In [8]:
def split_classes(s):
    no_slashes = s.replace('/', '.')
    classes = no_slashes.split(',')
    classes_with_slashes = [c.replace('.', '/') for c in classes]
    #classes_valid = [c for c in classes if len(c) <= 9]
    return sorted(classes_with_slashes)

In [9]:
df['cpc_classes'] = df['cpcs'].map(split_classes)

In [10]:
df['ipc_classes'] = df['ipcs'].map(split_classes)

In [11]:
def parse_ipcs(ipcs_list):
    first_letters = set()
    three_letters = set()
    four_letters = set()
    before_slash = set()
    for ipc in ipcs_list:
        if len(ipc) > 0:
            first_letters.add(ipc[0])
            three_letters.add(ipc[0:3])
            four_letters.add(ipc[0:4])
            s = ipc[0:ipc.find('/')]
            if len(s) >= 5 and len(s) <=6:
                before_slash.add(s)
    
    return sorted(list(first_letters)), sorted(list(three_letters)), sorted(list(four_letters)), sorted(list(before_slash))

In [12]:
df['first_letters'], df['three_letters'], df['four_letters'], df['before_slash'] = zip(*df['ipc_classes'].map(parse_ipcs))
df.head()

,pub_num,publication_number,country_code,family_id,priority_date,title_text,abstract_text,claims_text,refs,cpcs,ipcs,assignees_harmonized,ExpansionLevel,text_prep,tokens,year,month,day,cpc_classes,ipc_classes,first_letters,three_letters,four_letters,before_slash
0,4104187,US-4104187-A,US,24713901,19760412,Composition and method treating soft contact lenses at elevated temperatures,Method and compositions are provided for elevated temperature disinfection of soft contact lense...,What is claimed is: \n \n 1. A soft lens thermal disinfecting and cleaning solution co...,"US-3171752-A,US-4013576-A,US-2089212-A,US-3888782-A,US-3908680-A,US-3882036-A,,US-4046706-A","C11D1/523,C11D3/0078,A61L12/148","C11D17/00,G02C7/04,C11D1/66,C11D1/52,G02C13/00,C11D1/90,C11D3/20,A61L12/14,C11D3/00,C11D10/02,C1...",BARNES HIND PHARM INC,Seed,Composition and method treating soft contact lenses at elevated temperatures Method and composit...,"[composition, treating, soft, contact, lenses, elevated, temperatures, compositions, provided, e...",1976,4,12,"[A61L12/148, C11D1/523, C11D3/0078]","[A61L12/14, C11D1/52, C11D1/66, C11D1/90, C11D10/02, C11D17/00, C11D3/00, C11D3/20, C11D3/48, G0...","[A, C, G]","[A61, C11, G02]","[A61L, C11D, G02C]","[A61L12, C11D1, C11D10, C11D17, C11D3, G02C13, G02C7]"
1,6000534,US-6000534-A,US,24806490,19960816,Contact lens disinfecting device and disinfection system,The present invention provides a device for disinfecting and storing a contact lens comprising a...,I claim: \n \n 1. A device for disinfecting and storing a contact lens comprising: a c...,"US-4750610-A,US-4889693-A,US-5143104-A,US-5353949-A,US-5283053-A,US-4136796-A,US-4637919-A,US-47...","A45C11/005,Y10S134/901,A61L12/128","A61L12/12,G02C13/00,A61L2/18,A45C11/00",ALLERGAN SALES INC,Seed,Contact lens disinfecting device and disinfection system The present invention provides a device...,"[contact, lens, disinfecting, device, disinfection, system, device, disinfecting, storing, conta...",1996,8,16,"[A45C11/005, A61L12/128, Y10S134/901]","[A45C11/00, A61L12/12, A61L2/18, G02C13/00]","[A, G]","[A45, A61, G02]","[A45C, A61L, G02C]","[A45C11, A61L12, A61L2, G02C13]"
2,7281794,US-7281794-B2,US,25536666,20011116,Disposable ophthalmic lens,"An ophthalmic lens with high definition, wide field of view and high magnification has, in one e...",What is claimed and desired to be secured by Letters Patent is: \n \n 1. An ophthalmic...,"US-6471396-B2,US-4921205-A,US-4613379-A,US-5706073-A,US-4728183-A",A61B3/125,"G02C7/04,A61B3/125",HEACOCK GREGORY L,Seed,"Disposable ophthalmic lens An ophthalmic lens with high definition, wide field of view and high ...","[disposable, ophthalmic, lens, ophthalmic, lens, high, definition, wide, view, high, magnificati...",2001,11,16,[A61B3/125],"[A61B3/125, G02C7/04]","[A, G]","[A61, G02]","[A61B, G02C]","[A61B3, G02C7]"
3,8693104,US-8693104-B2,US,44651943,20100824,Lens with compound linear-convex meniscus wall,The present invention relates generally to an arcuate liquid meniscus lens with a meniscus wall....,The invention claimed is: \n \n 1. An optical lens comprising:\n a front curve lens co...,"WO-2008062067-A1,US-7724444-B2,US-7311398-B2,WO-2004099844-A1,US-7327523-B2,WO-2005069043-A1,US-...","G02C7/085,G02B26/004,A61F2/1635,G02C7/08,G02B3/14,G02C7/04","G02B3/12,G02B1/06","KERNICK EDWARD R,RIALL JAMES DANIEL,JOHNSON & JOHNSON VISION CARE,OTTS DANIEL B,TONER ADAM,PUGH ...",Seed,Lens with compound linear-convex meniscus wall The present invention relates generally to an arc...,"[lens, compound, linear, convex, meniscus, wall, arcuate, liquid, meniscus, lens, meniscus, wall...",2010,8,24,"[A61F2/1635, G02B26/004, G02B3/14, G02C7/04, G02C7/08, G02C7/085]","[G02B1/06, G02B3/12]",[G],[G02],[G02B],"[G02B1, G02B3]"
4,8215770,US-8215770-B2,US,40337739,20070223,Ophthalmic dynamic aperture,Embodiments of the present invention relate to an electro-active element having a dynamic apertu...,"1. An ophthalmic device, comprising:\n an electro-active eleme

In [13]:
df['references'] = df['refs'].map(lambda r: list(filter(None, r.split(','))))
df.head()

,pub_num,publication_number,country_code,family_id,priority_date,title_text,abstract_text,claims_text,refs,cpcs,ipcs,assignees_harmonized,ExpansionLevel,text_prep,tokens,year,month,day,cpc_classes,ipc_classes,first_letters,three_letters,four_letters,before_slash,references
0,4104187,US-4104187-A,US,24713901,19760412,Composition and method treating soft contact lenses at elevated temperatures,Method and compositions are provided for elevated temperature disinfection of soft contact lense...,What is claimed is: \n \n 1. A soft lens thermal disinfecting and cleaning solution co...,"US-3171752-A,US-4013576-A,US-2089212-A,US-3888782-A,US-3908680-A,US-3882036-A,,US-4046706-A","C11D1/523,C11D3/0078,A61L12/148","C11D17/00,G02C7/04,C11D1/66,C11D1/52,G02C13/00,C11D1/90,C11D3/20,A61L12/14,C11D3/00,C11D10/02,C1...",BARNES HIND PHARM INC,Seed,Composition and method treating soft contact lenses at elevated temperatures Method and composit...,"[composition, treating, soft, contact, lenses, elevated, temperatures, compositions, provided, e...",1976,4,12,"[A61L12/148, C11D1/523, C11D3/0078]","[A61L12/14, C11D1/52, C11D1/66, C11D1/90, C11D10/02, C11D17/00, C11D3/00, C11D3/20, C11D3/48, G0...","[A, C, G]","[A61, C11, G02]","[A61L, C11D, G02C]","[A61L12, C11D1, C11D10, C11D17, C11D3, G02C13, G02C7]","[US-3171752-A, US-4013576-A, US-2089212-A, US-3888782-A, US-3908680-A, US-3882036-A, US-4046706-A]"
1,6000534,US-6000534-A,US,24806490,19960816,Contact lens disinfecting device and disinfection system,The present invention provides a device for disinfecting and storing a contact lens comprising a...,I claim: \n \n 1. A device for disinfecting and storing a contact lens comprising: a c...,"US-4750610-A,US-4889693-A,US-5143104-A,US-5353949-A,US-5283053-A,US-4136796-A,US-4637919-A,US-47...","A45C11/005,Y10S134/901,A61L12/128","A61L12/12,G02C13/00,A61L2/18,A45C11/00",ALLERGAN SALES INC,Seed,Contact lens disinfecting device and disinfection system The present invention provides a device...,"[contact, lens, disinfecting, device, disinfection, system, device, disinfecting, storing, conta...",1996,8,16,"[A45C11/005, A61L12/128, Y10S134/901]","[A45C11/00, A61L12/12, A61L2/18, G02C13/00]","[A, G]","[A45, A61, G02]","[A45C, A61L, G02C]","[A45C11, A61L12, A61L2, G02C13]","[US-4750610-A, US-4889693-A, US-5143104-A, US-5353949-A, US-5283053-A, US-4136796-A, US-4637919-..."
2,7281794,US-7281794-B2,US,25536666,20011116,Disposable ophthalmic lens,"An ophthalmic lens with high definition, wide field of view and high magnification has, in one e...",What is claimed and desired to be secured by Letters Patent is: \n \n 1. An ophthalmic...,"US-6471396-B2,US-4921205-A,US-4613379-A,US-5706073-A,US-4728183-A",A61B3/125,"G02C7/04,A61B3/125",HEACOCK GREGORY L,Seed,"Disposable ophthalmic lens An ophthalmic lens with high definition, wide field of view and high ...","[disposable, ophthalmic, lens, ophthalmic, lens, high, definition, wide, view, high, magnificati...",2001,11,16,[A61B3/125],"[A61B3/125, G02C7/04]","[A, G]","[A61, G02]","[A61B, G02C]","[A61B3, G02C7]","[US-6471396-B2, US-4921205-A, US-4613379-A, US-5706073-A, US-4728183-A]"
3,8693104,US-8693104-B2,US,44651943,20100824,Lens with compound linear-convex meniscus wall,The present invention relates generally to an arcuate liquid meniscus lens with a meniscus wall....,The invention claimed is: \n \n 1. An optical lens comprising:\n a front curve lens co...,"WO-2008062067-A1,US-7724444-B2,US-7311398-B2,WO-2004099844-A1,US-7327523-B2,WO-2005069043-A1,US-...","G02C7/085,G02B26/004,A61F2/1635,G02C7/08,G02B3/14,G02C7/04","G02B3/12,G02B1/06","KERNICK EDWARD R,RIALL JAMES DANIEL,JOHNSON & JOHNSON VISION CARE,OTTS DANIEL B,TONER ADAM,PUGH ...",Seed,Lens with compound linear-convex meniscus wall The present invention relates generally to an arc...,"[lens, compound, linear, convex, meniscus, wall, arcuate, liquid, meniscus, lens, meniscus, wall...",2010,8,24,"[A61F2/1635, G02B26/004, G02B3/14, G02C7/04, G02C7/08, G02C7/085]","[G02

In [14]:
def find_backwards_references(publication_number):
    mask = df.references.apply(lambda x: publication_number in x)
    backward_references = df[mask]  
    result = pd.unique(backward_references["publication_number"])
    return result

df['referenced_by'] = df['publication_number'].map(lambda n: find_backwards_references(n))
df.head()

,pub_num,publication_number,country_code,family_id,priority_date,title_text,abstract_text,claims_text,refs,cpcs,ipcs,assignees_harmonized,ExpansionLevel,text_prep,tokens,year,month,day,cpc_classes,ipc_classes,first_letters,three_letters,four_letters,before_slash,references,referenced_by
0,4104187,US-4104187-A,US,24713901,19760412,Composition and method treating soft contact lenses at elevated temperatures,Method and compositions are provided for elevated temperature disinfection of soft contact lense...,What is claimed is: \n \n 1. A soft lens thermal disinfecting and cleaning solution co...,"US-3171752-A,US-4013576-A,US-2089212-A,US-3888782-A,US-3908680-A,US-3882036-A,,US-4046706-A","C11D1/523,C11D3/0078,A61L12/148","C11D17/00,G02C7/04,C11D1/66,C11D1/52,G02C13/00,C11D1/90,C11D3/20,A61L12/14,C11D3/00,C11D10/02,C1...",BARNES HIND PHARM INC,Seed,Composition and method treating soft contact lenses at elevated temperatures Method and composit...,"[composition, treating, soft, contact, lenses, elevated, temperatures, compositions, provided, e...",1976,4,12,"[A61L12/148, C11D1/523, C11D3/0078]","[A61L12/14, C11D1/52, C11D1/66, C11D1/90, C11D10/02, C11D17/00, C11D3/00, C11D3/20, C11D3/48, G0...","[A, C, G]","[A61, C11, G02]","[A61L, C11D, G02C]","[A61L12, C11D1, C11D10, C11D17, C11D3, G02C13, G02C7]","[US-3171752-A, US-4013576-A, US-2089212-A, US-3888782-A, US-3908680-A, US-3882036-A, US-4046706-A]","[US-4259202-A, US-2006241001-A1, US-4533399-A, US-4678698-A, US-4622258-A, US-4485029-A]"
1,6000534,US-6000534-A,US,24806490,19960816,Contact lens disinfecting device and disinfection system,The present invention provides a device for disinfecting and storing a contact lens comprising a...,I claim: \n \n 1. A device for disinfecting and storing a contact lens comprising: a c...,"US-4750610-A,US-4889693-A,US-5143104-A,US-5353949-A,US-5283053-A,US-4136796-A,US-4637919-A,US-47...","A45C11/005,Y10S134/901,A61L12/128","A61L12/12,G02C13/00,A61L2/18,A45C11/00",ALLERGAN SALES INC,Seed,Contact lens disinfecting device and disinfection system The present invention provides a device...,"[contact, lens, disinfecting, device, disinfection, system, device, disinfecting, storing, conta...",1996,8,16,"[A45C11/005, A61L12/128, Y10S134/901]","[A45C11/00, A61L12/12, A61L2/18, G02C13/00]","[A, G]","[A45, A61, G02]","[A45C, A61L, G02C]","[A45C11, A61L12, A61L2, G02C13]","[US-4750610-A, US-4889693-A, US-5143104-A, US-5353949-A, US-5283053-A, US-4136796-A, US-4637919-...",[]
2,7281794,US-7281794-B2,US,25536666,20011116,Disposable ophthalmic lens,"An ophthalmic lens with high definition, wide field of view and high magnification has, in one e...",What is claimed and desired to be secured by Letters Patent is: \n \n 1. An ophthalmic...,"US-6471396-B2,US-4921205-A,US-4613379-A,US-5706073-A,US-4728183-A",A61B3/125,"G02C7/04,A61B3/125",HEACOCK GREGORY L,Seed,"Disposable ophthalmic lens An ophthalmic lens with high definition, wide field of view and high ...","[disposable, ophthalmic, lens, ophthalmic, lens, high, definition, wide, view, high, magnificati...",2001,11,16,[A61B3/125],"[A61B3/125, G02C7/04]","[A, G]","[A61, G02]","[A61B, G02C]","[A61B3, G02C7]","[US-6471396-B2, US-4921205-A, US-4613379-A, US-5706073-A, US-4728183-A]",[]
3,8693104,US-8693104-B2,US,44651943,20100824,Lens with compound linear-convex meniscus wall,The present invention relates generally to an arcuate liquid meniscus lens with a meniscus wall....,The invention claimed is: \n \n 1. An optical lens comprising:\n a front curve lens co...,"WO-2008062067-A1,US-7724444-B2,US-7311398-B2,WO-2004099844-A1,US-7327523-B2,WO-2005069043-A1,US-...","G02C7/085,G02B26/004,A61F2/1635,G02C7/08,G02B3/14,G02C7/04","G02B3/12,G02B1/06","KERNICK EDWARD R,RIALL JAMES DANIEL,JOHNSON & JOHNSON VISION CARE,OTTS DANIEL B,TONER ADAM,PUGH ...",Seed,Lens with compound linear-convex meniscus wall The present invention relates generally to an arc...,"[lens, compound, linear, convex, meniscus, wall, arcuate, liquid, meniscu

In [15]:
from unidecode import unidecode

df['raw_assignees'] = df['assignees_harmonized'].map(lambda r: unidecode(r).split(','))

In [16]:
df.drop(columns = ['assignees_harmonized', 'refs', 'cpcs', 'ipcs', 'ExpansionLevel'], inplace = True)

In [17]:
df.head()

,pub_num,publication_number,country_code,family_id,priority_date,title_text,abstract_text,claims_text,text_prep,tokens,year,month,day,cpc_classes,ipc_classes,first_letters,three_letters,four_letters,before_slash,references,referenced_by,raw_assignees
0,4104187,US-4104187-A,US,24713901,19760412,Composition and method treating soft contact lenses at elevated temperatures,Method and compositions are provided for elevated temperature disinfection of soft contact lense...,What is claimed is: \n \n 1. A soft lens thermal disinfecting and cleaning solution co...,Composition and method treating soft contact lenses at elevated temperatures Method and composit...,"[composition, treating, soft, contact, lenses, elevated, temperatures, compositions, provided, e...",1976,4,12,"[A61L12/148, C11D1/523, C11D3/0078]","[A61L12/14, C11D1/52, C11D1/66, C11D1/90, C11D10/02, C11D17/00, C11D3/00, C11D3/20, C11D3/48, G0...","[A, C, G]","[A61, C11, G02]","[A61L, C11D, G02C]","[A61L12, C11D1, C11D10, C11D17, C11D3, G02C13, G02C7]","[US-3171752-A, US-4013576-A, US-2089212-A, US-3888782-A, US-3908680-A, US-3882036-A, US-4046706-A]","[US-4259202-A, US-2006241001-A1, US-4533399-A, US-4678698-A, US-4622258-A, US-4485029-A]",[BARNES HIND PHARM INC]
1,6000534,US-6000534-A,US,24806490,19960816,Contact lens disinfecting device and disinfection system,The present invention provides a device for disinfecting and storing a contact lens comprising a...,I claim: \n \n 1. A device for disinfecting and storing a contact lens comprising: a c...,Contact lens disinfecting device and disinfection system The present invention provides a device...,"[contact, lens, disinfecting, device, disinfection, system, device, disinfecting, storing, conta...",1996,8,16,"[A45C11/005, A61L12/128, Y10S134/901]","[A45C11/00, A61L12/12, A61L2/18, G02C13/00]","[A, G]","[A45, A61, G02]","[A45C, A61L, G02C]","[A45C11, A61L12, A61L2, G02C13]","[US-4750610-A, US-4889693-A, US-5143104-A, US-5353949-A, US-5283053-A, US-4136796-A, US-4637919-...",[],[ALLERGAN SALES INC]
2,7281794,US-7281794-B2,US,25536666,20011116,Disposable ophthalmic lens,"An ophthalmic lens with high definition, wide field of view and high magnification has, in one e...",What is claimed and desired to be secured by Letters Patent is: \n \n 1. An ophthalmic...,"Disposable ophthalmic lens An ophthalmic lens with high definition, wide field of view and high ...","[disposable, ophthalmic, lens, ophthalmic, lens, high, definition, wide, view, high, magnificati...",2001,11,16,[A61B3/125],"[A61B3/125, G02C7/04]","[A, G]","[A61, G02]","[A61B, G02C]","[A61B3, G02C7]","[US-6471396-B2, US-4921205-A, US-4613379-A, US-5706073-A, US-4728183-A]",[],[HEACOCK GREGORY L]
3,8693104,US-8693104-B2,US,44651943,20100824,Lens with compound linear-convex meniscus wall,The present invention relates generally to an arcuate liquid meniscus lens with a meniscus wall....,The invention claimed is: \n \n 1. An optical lens comprising:\n a front curve lens co...,Lens with compound linear-convex meniscus wall The present invention relates generally to an arc...,"[lens, compound, linear, convex, meniscus, wall, arcuate, liquid, meniscus, lens, meniscus, wall...",2010,8,24,"[A61F2/1635, G02B26/004, G02B3/14, G02C7/04, G02C7/08, G02C7/085]","[G02B1/06, G02B3/12]",[G],[G02],[G02B],"[G02B1, G02B3]","[WO-2008062067-A1, US-7724444-B2, US-7311398-B2, WO-2004099844-A1, US-7327523-B2, WO-2005069043-...",[],"[KERNICK EDWARD R, RIALL JAMES DANIEL, JOHNSON & JOHNSON VISION CARE, OTTS DANIEL B, TONER ADAM,..."
4,8215770,US-8215770-B2,US,40337739,20070223,Ophthalmic dynamic aperture,Embodiments of the present invention relate to an electro-active element having a dynamic apertu...,"1. An ophthalmic device, comprising:\n an electro-active element comprising a mostly transparent...",Ophthalmic dynamic aperture Embodiments of the present invention relate to an electro-active ele...,"[ophthalmic, dynamic, aperture, embodiments, electro, active, element, dynamic, aperture, electr..

### Merge assignees with similar names

In [18]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def dedupe_fixed(contains_dupes, threshold=70, scorer=fuzz.token_set_ratio):
    """This convenience function takes a list of strings containing duplicates and uses fuzzy matching to identify
    and remove duplicates. Specifically, it uses the process.extract to identify duplicates that
    score greater than a user defined threshold. Then, it looks for the longest item in the duplicate list
    since we assume this item contains the most entity information and returns that. It breaks string
    length ties on an alphabetical sort.
    Note: as the threshold DECREASES the number of duplicates that are found INCREASES. This means that the
        returned deduplicated list will likely be shorter. Raise the threshold for fuzzy_dedupe to be less
        sensitive.
    Args:
        contains_dupes: A list of strings that we would like to dedupe.
        threshold: the numerical value (0,100) point at which we expect to find duplicates.
            Defaults to 70 out of 100
        scorer: Optional function for scoring matches between the query and
            an individual processed choice. This should be a function
            of the form f(query, choice) -> int.
            By default, fuzz.token_set_ratio() is used and expects both query and
            choice to be strings.
    Returns:
        A deduplicated list. For example:
            In: contains_dupes = ['Frodo Baggin', 'Frodo Baggins', 'F. Baggins', 'Samwise G.', 'Gandalf', 'Bilbo Baggins']
            In: fuzzy_dedupe(contains_dupes)
            Out: ['Frodo Baggins', 'Samwise G.', 'Bilbo Baggins', 'Gandalf']
        """

    extractor = []
    processed = []

    # iterate over items in *contains_dupes*
    for item in contains_dupes:
        if item not in processed and item not in extractor:
            # return all duplicate matches found
            matches = process.extract(item, contains_dupes, limit=None, scorer=scorer)
            # filter matches based on the threshold
            filtered = [x for x in matches if x[1] >= threshold]
            # if there is only 1 item in *filtered*, no duplicates were found so append to *extracted*
            if len(filtered) == 1:
                extractor.append(filtered[0][0])

            else:
                # alpha sort
                filtered = sorted(filtered, key=lambda x: x[0])
                # length sort
                filter_sort = sorted(filtered, key=lambda x: len(x[0]))
                #filter_sort = sorted(filtered, key=lambda x: len(x[0]), reverse=True)
                # take first item as our 'canonical example'
                extractor.append(filter_sort[0][0])
                
            processed.append([i[0] for i in filtered])

    # uniquify *extractor* list
    
    set_extractor = set(extractor)
    extractor = list(set_extractor)

    # check that extractor differs from contain_dupes (e.g. duplicates were found)
    # if not, then return the original list
    if len(extractor) == len(contains_dupes):
        return contains_dupes
    else:
        return extractor

In [19]:
assignees = [item for sublist in df['raw_assignees'].values for item in sublist]
assignees_set = set(assignees)
print(len(assignees_set))
#assignees_set

1482


In [20]:
threshold = 88 #hair dryer

#scorer = fuzz.token_sort_ratio
scorer = fuzz.token_set_ratio

deduplicated = sorted(list(dedupe_fixed(assignees_set, threshold=threshold, scorer=scorer)))
print(len(deduplicated))

1277


In [21]:
deduplicated_dictionary = {}
for d in deduplicated:
    matches = process.extractBests(d, assignees_set, score_cutoff=threshold, scorer=scorer)
    deduplicated_dictionary[d] = matches
    
#for k, v in deduplicated_dictionary.items():
#    if len(v) > 1:
#        print(k, v)

In [22]:
%%time

def disambiguate_assignees(assignee_list):
    result = []
    for assignee in assignee_list:
        match = process.extractOne(assignee, deduplicated)
        result.append(match[0])
    return list(set(result))

df['assignees'] = df['raw_assignees'].map(lambda x: disambiguate_assignees(x))
df.head()

Wall time: 4min 23s


## Preprocessing of tokens to labels with TF-IDF

### Text Representation
We will use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">sklearn.feature_extraction.text.TfidfVectorizer</a> to calculate a tf-idf vector for each document.<br>
Therefore, we need to define the following model parameters:

1. **sublinear_tf:** is set to True to use a logarithmic form for frequency.
2. **min_df:**  is the minimum number of documents a word must occur in (our case: more than 10).
3. **max_df:** is the maximum number of documents a word is allowed to appear in (our case: less than 30% of the documents)
4. **norm:** is set to l2, to ensure all our feature vectors have a euclidian norm of 1.
5. **ngram_range:**  is set to (1, 2) to indicate that we want to consider unigrams and bigrams.

In [23]:
df["token_string"] = df["tokens"].apply(lambda x: " ".join(x))

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=10, max_df=0.2, norm='l2', encoding='latin-1', smooth_idf=True, use_idf=True, ngram_range=(1, 2))
term_doc_matrix = vectorizer.fit_transform(df["token_string"])

feature_array = np.array(vectorizer.get_feature_names())

In [25]:
def get_sorted_terms(x, vectorizer):
    transformed = vectorizer.transform([x])
    tfidf_sorting = np.argsort(transformed.toarray()).flatten()[::-1]
    sorted_terms = feature_array[tfidf_sorting]
    return sorted_terms

In [26]:
df["terms"] = df["token_string"].apply(lambda x: get_sorted_terms(x, vectorizer))

In [27]:
df.drop(columns=["token_string", 'text_prep', 'cpc_classes'], inplace=True)
df.head()

,pub_num,publication_number,country_code,family_id,priority_date,title_text,abstract_text,claims_text,tokens,year,month,day,ipc_classes,first_letters,three_letters,four_letters,before_slash,references,referenced_by,raw_assignees,assignees,terms
0,4104187,US-4104187-A,US,24713901,19760412,Composition and method treating soft contact lenses at elevated temperatures,Method and compositions are provided for elevated temperature disinfection of soft contact lense...,What is claimed is: \n \n 1. A soft lens thermal disinfecting and cleaning solution co...,"[composition, treating, soft, contact, lenses, elevated, temperatures, compositions, provided, e...",1976,4,12,"[A61L12/14, C11D1/52, C11D1/66, C11D1/90, C11D10/02, C11D17/00, C11D3/00, C11D3/20, C11D3/48, G0...","[A, C, G]","[A61, C11, G02]","[A61L, C11D, G02C]","[A61L12, C11D1, C11D10, C11D17, C11D3, G02C13, G02C7]","[US-3171752-A, US-4013576-A, US-2089212-A, US-3888782-A, US-3908680-A, US-3882036-A, US-4046706-A]","[US-4259202-A, US-2006241001-A1, US-4533399-A, US-4678698-A, US-4622258-A, US-4485029-A]",[BARNES HIND PHARM INC],[BARNES HIND INC],"[physiologically acceptable, disinfecting cleaning, physiologically, fatty acid, fatty, ethylene..."
1,6000534,US-6000534-A,US,24806490,19960816,Contact lens disinfecting device and disinfection system,The present invention provides a device for disinfecting and storing a contact lens comprising a...,I claim: \n \n 1. A device for disinfecting and storing a contact lens comprising: a c...,"[contact, lens, disinfecting, device, disinfection, system, device, disinfecting, storing, conta...",1996,8,16,"[A45C11/00, A61L12/12, A61L2/18, G02C13/00]","[A, G]","[A45, A61, G02]","[A45C, A61L, G02C]","[A45C11, A61L12, A61L2, G02C13]","[US-4750610-A, US-4889693-A, US-5143104-A, US-5353949-A, US-5283053-A, US-4136796-A, US-4637919-...",[],[ALLERGAN SALES INC],[ADVANCED CORNEAL SYSTEMS INC],"[lens holding, cap, internal surface, container cap, internal, impermeable, open end, container,..."
2,7281794,US-7281794-B2,US,25536666,20011116,Disposable ophthalmic lens,"An ophthalmic lens with high definition, wide field of view and high magnification has, in one e...",What is claimed and desired to be secured by Letters Patent is: \n \n 1. An ophthalmic...,"[disposable, ophthalmic, lens, ophthalmic, lens, high, definition, wide, view, high, magnificati...",2001,11,16,"[A61B3/125, G02C7/04]","[A, G]","[A61, G02]","[A61B, G02C]","[A61B3, G02C7]","[US-6471396-B2, US-4921205-A, US-4613379-A, US-5706073-A, US-4728183-A]",[],[HEACOCK GREGORY L],[HEACOCK GREGORY L],"[image lens, lens image, holder, lens mounted, lens intermediate, contact eye, image, rays, inte..."
3,8693104,US-8693104-B2,US,44651943,20100824,Lens with compound linear-convex meniscus wall,The present invention relates generally to an arcuate liquid meniscus lens with a meniscus wall....,The invention claimed is: \n \n 1. An optical lens comprising:\n a front curve lens co...,"[lens, compound, linear, convex, meniscus, wall, arcuate, liquid, meniscus, lens, meniscus, wall...",2010,8,24,"[G02B1/06, G02B3/12]",[G],[G02],[G02B],"[G02B1, G02B3]","[WO-2008062067-A1, US-7724444-B2, US-7311398-B2, WO-2004099844-A1, US-7327523-B2, WO-2005069043-...",[],"[KERNICK EDWARD R, RIALL JAMES DANIEL, JOHNSON & JOHNSON VISION CARE, OTTS DANIEL B, TONER ADAM,...","[SNOOK SHARIKA, RIALL JAMES D, PUGH RANDALL B, OTTS DANIEL B, TONER ADAM, KERNICK EDWARD, JOHNSO...","[curve lens, meniscus, optical lens, electrical charge, meniscus wall, lens interior, interior s..."
4,8215770,US-8215770-B2,US,40337739,20070223,Ophthalmic dynamic aperture,Embodiments of the present invention relate to an electro-active element having a dynamic apertu...,"1. An ophthalmic device, comprising:\n an electro-active element comprising a mostly transparent...","[ophthalmic, dynamic, aperture, embodiments, electro, active, element, dynamic, aperture, electr...",2007,2,23,"[A61F2/16, G02C7/04]","[A, G]","[A61, G02]","[A61F, G02C]","[A61F2, G02C7

In [28]:
with open(src_dir + "/data/" + seed_name + "/terms_attributes.pkl", 'wb') as outfile:
    pickle.dump(df, outfile)